In [ ]:
!pip install --upgrade --no-cache-dir setuptools==57.5.0
!pip install python-dotenv gdal==3.6.4

Load env vars file

In [2]:
%reload_ext dotenv
%dotenv .env

In [ ]:
import os
from dotenv import load_dotenv
import concurrent
import requests
from osgeo import gdal
from lib.download_dem_data import download_tile

load_dotenv("local.env")
username = os.getenv("EARTHDATA_USERNAME")
password = os.getenv("EARTHDATA_PASSWORD")
tiles_url_root = os.getenv("TILES_URL_ROOT") # The url prefix to use for the tile file names
tiles_file_path = os.getenv("TILES_FILE_PATH") # The path to a file containing a list of file names
tiles_file = open(tiles_file_path) # The file containing a list of file names
tile_file_names = tiles_file.readlines() # The list of file names

def run():
    output_path = "data/earthdata_tiles"
    os.makedirs(output_path, exist_ok=True)
    gdal.SetConfigOption('GDAL_PAM_ENABLED', 'NO')
    
    with requests.Session() as session:
        session.auth = (username, password)

        with concurrent.futures.ProcessPoolExecutor() as process_pool:
            futures = []
            for tile_file_name in tile_file_names:
                download_url = os.path.join(tiles_url_root, tile_file_name.strip())
                destination_path = os.path.join(output_path, tile_file_name.strip())
                futures.append(process_pool.submit(download_tile, download_url, destination_path, session))

            print(f'Downloading {len(futures)} files...')
            for future in concurrent.futures.as_completed(futures):
                print(future.result())
        print('Done!')

if __name__ == "__main__":
    run()
